<a href="https://colab.research.google.com/github/marreapato/Deep_Learning_Course/blob/main/Reading_MIT_Indoor_Scene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import os

# Define the URL of the dataset
url = "http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar"

# Define the directory where you want to save the dataset
save_dir = "mit_indoor_scene_dataset"

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define the path where you want to save the downloaded file
save_path = os.path.join(save_dir, "indoorCVPR_09.tar.gz")

# Send an HTTP GET request to download the dataset
response = requests.get(url, stream=True)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the file in binary write mode and write the content of the response
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)

    print(f"Downloaded the dataset to {save_path}")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

# You can now extract the contents of the downloaded tar.gz file using Python's tarfile module if needed.


Downloaded the dataset to mit_indoor_scene_dataset/indoorCVPR_09.tar.gz


In [9]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For separating train and test sets
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
import numpy as np
import pathlib

In [15]:
from PIL import Image
import os
import tarfile
import os

# Define the path to an example image
image_path = "/content/mit_indoor_scene_dataset/indoorCVPR_09.tar.gz"



t = tarfile.open(image_path, 'r')
for member in t.getmembers():
    if ".jpg" in member.name:
        t.extract(member, "outdir")

print(os.listdir('outdir'))

['Images']
